# DSC 630 - Predictive Analytics
# Term Project - Summer 2024
# David Berberena | Brian Mann

In [1]:
# imported libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

## Milestone 2 - Data Selection and Project Proposal

In [ ]:
# datasets
url1 = 'www.example1.com/data'

### Step 1: ...

### Step 2: ...